## Homework 5
## Yue Yu yy1879
### Question 1


In [4]:
import pandas as pd
import numpy as np
import math
import scipy.optimize as optimize

In [5]:
fund1 = pd.read_excel('data.xls',sheetname = 'VBTLX',header=1)
fund2 = pd.read_excel('data.xls',sheetname = 'VWILX',header=1)
fund3 = pd.read_excel('data.xls',sheetname = 'VGPMX',header=1)
fund4 = pd.read_excel('data.xls',sheetname = 'VTSAX',header=1)
fund5 = pd.read_excel('data.xls',sheetname = 'VTRIX',header=1)
fund6 = pd.read_excel('data.xls',sheetname = 'VGSLX',header=1)
fund7 = pd.read_excel('data.xls',sheetname = 'VMRXX',header=1)
annualize_m = []
fundlist = [fund1,fund2,fund3,fund4,fund5,fund6,fund7]
def annualize(r,*args):
    q1, q2, q3, q4=args
    return (1+q1)*(1+q2)*(1+q3)*(1+q4)-(1+r)
for fund in fundlist:
    return_v = np.zeros(len(fund))
    for i in range(len(fund1)):
        x = optimize.fsolve(annualize,0.0000001,args=(fund['Q1'][2018-i],fund['Q2'][2018-i],fund['Q3'][2018-i],fund['Q4'][2018-i]))
        return_v[i]=x[0]
#         return_v[i]=fund['Q1'][2018-i]+fund['Q2'][2018-i]+fund['Q3'][2018-i]+fund['Q4'][2018-i]
    return_v=return_v.reshape(-1,1)
    annualize_m.append(return_v)
return_m = np.hstack(annualize_m)
mean_v=np.mean(return_m,axis=0)
pd.DataFrame(mean_v)

,0
0,0.039070
1,0.071004
2,0.012914
3,0.089385
4,0.037068
5,0.061481
6,0.011021


In [6]:
line1 = [1.0130,-0.0079,0.0203,-0.0146,-0.0084,0.0114]
line2 = [-0.0079,1.5142,0.5153,0.3572,0.4678,0.3617]
line3 = [0.0203,0.5153,2.1815,0.3348,0.4790,0.3960]
line4 = [-0.0146,0.3572,0.3348,1.3233,0.3374,0.3526]
line5 = [-0.0084,0.4678,0.4790,0.3374,1.4456,0.3467]
line6 = [0.0114,0.3617,0.3960,0.3526,0.3467,1.6877]
cov_m = np.matrix([line1,line2,line3,line4,line5,line6])
# cov_m.columns = ['Bond','Int_Growth','Metals','Stock','Int_Value','REIT']
pd.DataFrame(cov_m,columns = ['Bond','Int_Growth','Metals','Stock','Int_Value','REIT'])

,Bond,Int_Growth,Metals,Stock,Int_Value,REIT
0,1.0130,-0.0079,0.0203,-0.0146,-0.0084,0.0114
1,-0.0079,1.5142,0.5153,0.3572,0.4678,0.3617
2,0.0203,0.5153,2.1815,0.3348,0.4790,0.3960
3,-0.0146,0.3572,0.3348,1.3233,0.3374,0.3526
4,-0.0084,0.4678,0.4790,0.3374,1.4456,0.3467
5,0.0114,0.3617,0.3960,0.3526,0.3467,1.6877


In [7]:
inv_c = np.linalg.inv(cov_m)
pd.DataFrame(inv_c)

,0,1,2,3,4,5
0,0.987819,0.006401,-0.012562,0.013029,0.007011,-0.009259
1,0.006401,0.802247,-0.118060,-0.120375,-0.172275,-0.083737
2,-0.012562,-0.118060,0.530894,-0.057582,-0.108800,-0.064801
3,0.013029,-0.120375,-0.057582,0.863596,-0.115266,-0.117526
4,0.007011,-0.172275,-0.108800,-0.115266,0.830682,-0.084161
5,-0.009259,-0.083737,-0.064801,-0.117526,-0.084161,0.667578


In [8]:
def tangency_portfolio(m, C, Rf):
    def objective_func(weights):
        weight = np.append(weights, 1 - weights.sum())
        return - (weight @ m - Rf)/(math.sqrt(weight@C@weight))
    init = (np.ones(len(m)-1)/(len(m)-1))
    result = optimize.minimize(objective_func, init)
    if result.status == 0: 
        return np.append(result.x, 1-result.x.sum())
    else:
        print('Fail to find')
optimal_w = tangency_portfolio(mean_v[0:-1],cov_m,mean_v[-1])
pd.DataFrame(optimal_w)

,0
0,0.265794
1,0.276212
2,-0.157299
3,0.477836
4,-0.018303
5,0.155761


### Question 2
We followed the instructions in the Black Litterman paper to do all the constructions of input.

### Construction of P and Q (view vector and matrix)
First, according to the views, we can construct
$$Q = \begin{bmatrix}0.002\\0\\0\\0\end{bmatrix}$$
since P is matched to Q, we have 
$$P = \begin{bmatrix}0&1&0&-1&0&0\\1&0&0&0&0&0\\1&0&-1&0&0&0\\0&0&a&b&0&1\end{bmatrix}$$

Using market capitalization weighting scheme, $a,b$ in P is in proportion to $$\frac{\text{asset market cap}}{\text{total underperforming market cap}}$$

Hence, we have $a=\frac{1}{6}$, $b=\frac{5}{6}$.

Therefore, we finalize P as 
$$P = \begin{bmatrix}0&1&0&-1&0&0\\1&0&0&0&0&0\\1&0&-1&0&0&0\\0&0&\frac{1}{6}&\frac{5}{6}&0&1\end{bmatrix}$$

### Construction of $\tau$ and $\Omega$
$\tau$ is inversely proportional to relative weight. We could use $\frac{1}{\text{# of observations}}$ to estimate $\tau$.

Or we could set $\tau$ randomly, say $0.0025$, and then set 

$$\Omega = \begin{bmatrix}p_1 \sum p_1^T *\tau &0&0&0&0\\.. \\0&0&0&0&p_k \sum p_k^T *\tau\end{bmatrix}$$
where $p_k$ is the $k^{th}$ row vector in $P$.

In this way, $\tau$ becomes "irrelevant" in this model, only $\frac{\omega}{\tau}$ goes in the model, $\omega$ be the variance of the error terms in view.

### Construction of $\Pi$
$Pi$ is the implied equilibrium return factor in Black-Litterman model and can be expressed as 

$$\Pi = \lambda \Sigma w_{wkt}$$

with $\lambda$ be the risk aversion coefficients $\frac{E(r)-r_f}{\sigma^2}$, $\Sigma$ the covariance matrix, and $w_{mkt}$ be the market cap weights.


In [14]:
P= np.matrix([[0,1,0,-1,0,0],[1,0,0,0,0,0],[1,0,-1,0,0,0],[0,0,1/6,5/6,0,1]])
print('View matrix P:')
pd.DataFrame(P)

matrix P:


,0,1,2,3,4,5
0,0.0,1.0,0.000000,-1.000000,0.0,0.0
1,1.0,0.0,0.000000,0.000000,0.0,0.0
2,1.0,0.0,-1.000000,0.000000,0.0,0.0
3,0.0,0.0,0.166667,0.833333,0.0,1.0


In [16]:
Q=np.matrix([[0.002],[0],[0],[0]])
print('View vector Q:')
pd.DataFrame(Q)

View vector Q:


,0
0,0.002
1,0.000
2,0.000
3,0.000


In [29]:
tau = 0.0025
O = np.zeros((4,4))
for i in range(4):
    O[i][i] = P[i]@cov_m@P[i].T*tau
pd.DataFrame(O)

,0,1,2,3
0,0.005308,0.000000,0.000000,0.0000
1,0.000000,0.002532,0.000000,0.0000
2,0.000000,0.000000,0.007885,0.0000
3,0.000000,0.000000,0.000000,0.0087


In [35]:
w = np.matrix([[5],[2],[1],[5],[2],[3]])
lam = 1
pi = lam*cov_m@w
pd.DataFrame(cov_m@w)

,0
0,5.0139
1,7.3109
2,7.1336
3,9.3253
4,6.9909
5,8.6959


In [34]:
part1 = np.linalg.inv(tau*cov_m)+P.T@np.linalg.inv(O)@P
pd.DataFrame(part1)

,0,1,2,3,4,5
0,916.821277,2.560465,-131.852061,5.211455,2.804317,-3.703564
1,2.560465,509.302517,-47.223817,-236.553571,-68.910063,-33.494635
2,-131.852061,-47.223817,342.377717,-7.068146,-43.519926,-6.762852
3,5.211455,-236.553571,-7.068146,613.665180,-46.106493,48.777373
4,2.804317,-68.910063,-43.519926,-46.106493,332.272721,-33.664225
5,-3.703564,-33.494635,-6.762852,48.777373,-33.664225,381.976491


In [37]:
part2 = np.linalg.inv(tau*cov_m)@pi + P.T@np.linalg.inv(O)@Q
pd.DataFrame(part2)

matrix([[ 2000.,  2000.,  2000.,  2000.],
        [  800.,   800.,   800.,   800.],
        [  400.,   400.,   400.,   400.],
        [ 2000.,  2000.,  2000.,  2000.],
        [  800.,   800.,   800.,   800.],
        [ 1200.,  1200.,  1200.,  1200.]])

In [38]:
bl = np.linalg.inv(part1)@part2
pd.DataFrame(bl)

,0,1,2,3
0,2.675472,2.675472,2.675472,2.675472
1,5.355780,5.355780,5.355780,5.355780
2,3.765939,3.765939,3.765939,3.765939
3,5.452402,5.452402,5.452402,5.452402
4,5.095857,5.095857,5.095857,5.095857
5,3.456656,3.456656,3.456656,3.456656
